In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok
!pip install ffmpeg-python
!pip install pydub

In [ ]:
from flask import Flask, url_for, redirect, render_template, request, redirect
from flask_ngrok import run_with_ngrok
import os
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import soundfile
import librosa
from scipy.io.wavfile import write
import pickle
import soundfile as sf
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read
import io
import ffmpeg
from IPython.display import Javascript
from google.colab import output
from io import BytesIO
from pydub import AudioSegment

TEMPLATE_FOLDER = '/content/drive/Shareddrives/Hx-corps/수진/Vegita-master/Flask Web Server/templates' 
STATIC_FOLDER = '/content/drive/Shareddrives/Hx-corps/수진/Vegita-master/Flask Web Server/static' 
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)   

# 저장한 모델 불러오기
%cd '/content/drive/Shareddrives/Hx-corps/web_app'
loaded_model = pickle.load(open('/content/drive/Shareddrives/Hx-corps/web_app/second_pickle', 'rb'))

@app.route("/", methods=['GET', 'POST'])
def index():
  if request.method == 'GET':
    return render_template('index_test_sujin3.html', emotion="img/question.png")

  if request.method == 'POST': 
    print ("FORM DATA RECEIVED")
    
    if request.form['start'] == 'start':
      
      from IPython.display import Javascript
      from google.colab import output
      from base64 import b64decode

      AUDIO_HTML = """
      <script>
      var my_div = document.createElement("DIV");
      var my_p = document.createElement("P");
      var my_btn = document.createElement("BUTTON");
      var t = document.createTextNode("Press to start recording");

      my_btn.appendChild(t);
      //my_p.appendChild(my_btn);
      my_div.appendChild(my_btn);
      document.body.appendChild(my_div);

      var base64data = 0;
      var reader;
      var recorder, gumStream;
      var recordButton = my_btn;

      var handleSuccess = function(stream) {
        gumStream = stream;
        var options = {
          //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
          mimeType : 'audio/webm;codecs=opus'
          //mimeType : 'audio/webm;codecs=pcm'
        };            
        //recorder = new MediaRecorder(stream, options);
        recorder = new MediaRecorder(stream);
        recorder.ondataavailable = function(e) {            
          var url = URL.createObjectURL(e.data);
          var preview = document.createElement('audio');
          preview.controls = true;
          preview.src = url;
          document.body.appendChild(preview);

          reader = new FileReader();
          reader.readAsDataURL(e.data); 
          reader.onloadend = function() {
            base64data = reader.result;
            //console.log("Inside FileReader:" + base64data);
          }
        };
        recorder.start();
        };

      recordButton.innerText = "Recording... press to stop";

      navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


      function toggleRecording() {
        if (recorder && recorder.state == "recording") {
            recorder.stop();
            gumStream.getAudioTracks()[0].stop();
            recordButton.innerText = "Saving the recording... pls wait!"
        }
      }

      // https://stackoverflow.com/a/951057
      function sleep(ms) {
        return new Promise(resolve => setTimeout(resolve, ms));
      }

      var data = new Promise(resolve=>{
      //recordButton.addEventListener("click", toggleRecording);
      recordButton.onclick = ()=>{
      toggleRecording()

      sleep(30).then(() => {
        // wait 30ms for the data to be available...
        // ideally this should use something like await...
        //console.log("Inside data:" + base64data)
        resolve(base64data.toString())

      });

      }
      });
            
      </script>
      """
      display(HTML(AUDIO_HTML))
      data = eval_js("data")
      binary = b64decode(data.split(',')[1])
      process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
      )
      output, err = process.communicate(input=binary)
      
      riff_chunk_size = len(output) - 8
      # Break up the chunk size into four bytes, held in b.
      q = riff_chunk_size
      b = []
      for i in range(4):
          q, r = divmod(q, 256)
          b.append(r)

      # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
      riff = output[:4] + bytes(b) + output[8:]

      sr, audio = read(io.BytesIO(riff))

      write("/content/drive/Shareddrives/Hx-corps/web_app/input.wav", sr, audio)

      # audio_stream = ffmpeg.input(audio)
      # audio_stream = ffmpeg.output(audio_stream, 'converted.mp3', **{'b:a': '16k'})

      # sound = AudioSegment.from_mp3("converted.mp3")
      # sound.export("/content/drive/Shareddrives/Hx-corps/web_app/huh2.wav", format="wav")

      test_file = '/content/drive/Shareddrives/Hx-corps/web_app/input.wav' #(수진)여기 initial.wav였는데 제가 바꿨어요
      # test_file = '/content/drive/Shareddrives/Hx-corps/web_app/huh2.wav'
      result = loaded_model.predict(test_file)
      result2="img/question.png"
      if result=="sad":
        result2="img/sadness.png"
      elif result=="angry":
        result2="img/angry.png"
      elif result=="fear":
        result2="img/fear.png"
      elif result=="disgust":
        result2="img/disgust.png"
      elif result=="neutral":
        result2="img/neutral.png"
      elif result=="ps":
        result2="img/surprise.png"
      else:
        result2="img/happiness.png"
    
      return render_template('index_test_sujin3.html', price=result, emotion=result2)

if __name__ == '__main__':
    app.run()
    #app.run(debut=True, threaded=True)

/content/drive/Shareddrives/Hx-corps/web_app
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator BaggingClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4bfc-34-91-112-119.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2022-01-03 04:25:54,319] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-3-e75fa7f73c07>", line 37, in index
    return render_template('index_test_sujin3.html', em

In [ ]:
# type in terminal
# ngrok http http://127.0.0.1:5000 --authtoken 20KEeuXyQTRtQ7mE1rjJ8SsNchu_33HJ3YnAhCqqaqVfygwwJ --region=us --hostname=iamyouare.ngrok.io

'''
if there's any error something like: collsion,,
try run again with other hostname below

iam.ngrok.io 유정
iam2.ngrok.io 지영
iam3.ngrok.io 수진
iam4.ngrok.io 혜영
iamyouareSER.ngrok.io


MUST DO:
terminal에서 실행 중단 : cntl + C
'''

